In [54]:
import json
from random import choices,shuffle

ds = json.load(open(r"data\question_with_parts_and_answerable_option.json", "r"))

def shuffle_list_of_dicts_based_on_a_key(ds: list[dict], min_len: int = 5, key: str = "can_i_answer") -> list[dict]:
    ds_true = [x for x in ds if x.get(key, True)]
    ds_false = [x for x in ds if not x.get(key, False)]
    min_len = min(len(ds_true), len(ds_false)) if not min_len else min_len

    ds_final = choices(ds_true, k=min_len) + choices(ds_false, k=min_len)
    shuffle(ds_final)
    return ds_final

example_output = shuffle_list_of_dicts_based_on_a_key(ds, 5)
example_input = [x["question"] for x in example_output]

In [57]:
json.dumps(example_output)

'[{"question": "What is the tallest mountain in the solar system and on which planet is it located?", "tasks": [], "can_i_answer": true}, {"question": "What are the must-visit tourist attractions near my current location?", "tasks": ["LOCATE: Identify the user\'s location", "MAPS: Find must-visit tourist attractions near the user\'s location"], "can_i_answer": false}, {"question": "Recommend online resources for learning Python programming.", "tasks": ["SEARCH: Retrieve online resources for learning Python programming"], "can_i_answer": false}, {"question": "Recommend online resources for learning Python programming.", "tasks": ["SEARCH: Retrieve online resources for learning Python programming"], "can_i_answer": false}, {"question": "What is the chemical formula for water and what are its three states of matter?", "tasks": [], "can_i_answer": true}, {"question": "Provide workout routines for effective weight loss in a month.", "tasks": [], "can_i_answer": true}, {"question": "What was

In [106]:
questions_breaker_prompt = f"""
You are an honest and smart assistant who can breakdown questions into 
simple steps/actions and also answer whether or not you can answer a question without external resources. 

You'll be given a list of questions as strings and you'll have to return a list of JSON objects with the following keys: question, tasks, can_i_answer
I have added a sample example input and output below

Example Input:
["What is the average income in my city?", "What are the latest findings on climate change?", "Which restaurants near me have the highest ratings?", "Compare prices of Dell and HP laptops with similar specifications.", "What are the ratings of Inception and Interstellar?", "Compare the weekend weather forecast for New York and Los Angeles.", "What are the must-visit tourist attractions near my current location?", "Find the nearest coffee shop from my current location.", "Give me directions to the nearest gas station in downtown.", "What is the average income for software engineers in San Francisco?", "Retrieve statistics on the prevalence of COVID-19 in my city.", "Show me the sales trends for iPhones over the last year.", "What was the GDP of the United States in the year 2000?", "Retrieve historical stock prices for Apple Inc.", "Tell me about significant historical events that happened on this day."]

Example Output:
{json.dumps(shuffle_list_of_dicts_based_on_a_key(ds, 5))}
"""


responses = call_openai_api(
    messages=[
        {"role": "system", "content": questions_breaker_prompt},
        {"role": "user", "content": json.dumps([x.get("question") for x in question_with_parts][:10])}
    ],
    temperature=0.7,
    n=1,
)

API: {'endpoint': 'https://east-us-2-llm-instance-01.openai.azure.com/', 'key1': '594a1155c8d948c2a8b535ab3adfa3bd', 'key2': '204722a1314f476f9620c5bafb5d5c20', 'region': 'eastus2', 'model_id': 'llm-chat-v4-128k-deploy-1', 'max_tokens': 4096}


In [268]:
from itertools import chain
import json
from datetime import datetime


def try_json_loads_list(text: str) -> list:
    try:
        return json.loads(text)
    except:
        return []


questions = list(
    chain(
        *[
            try_json_loads_list(remove_backticks(x.message.content))
            for x in responses.choices
        ]
    )
)
json.dump(
    questions,
    open(f"data/dumps/questions_{datetime.now().strftime('%Y%m%d%H%M%S')}.json", "w"),
)

In [6]:
import json
from random import choices
from itertools import chain
from pathlib import Path
from helpers.utils import try_json_load

QUESTIONS_PATH = Path("data/generated_questions/")

def get_random_questions(k=10) -> list[str]:
    return choices(list(chain(*choices([try_json_load(x) for x in QUESTIONS_PATH.rglob("*.json") if x.exists()], k=5))), k=10)

get_random_questions()

['What are the current best practices in cybersecurity for small businesses?',
 'Latest unemployment rates in top 5 global economies?',
 'Latest data on Arctic sea ice extent and thickness?',
 'What is the current status of the electric vehicle market?',
 'What are the newest developments in artificial intelligence research?',
 'Current best-selling authors of 2023?',
 'Recent breakthroughs in CRISPR gene-editing research?',
 'What are the top-grossing movies internationally in the past month?',
 'What is the latest publicly released iOS version by Apple?',
 'Major political elections and their outcomes around the world in 2023.']

In [264]:
questions_generator = f"""
You are an excellent question generator. 
You have to generate question that will be asked on a search engine to get latest data.

You will have to generate 20 random questions every time.
Your output should be a JSON array of strings. 

Here's how the output should look like:
{get_random_questions(10)}
"""

generated_questions = call_openai_api(messages=[{"role": "system", "content": questions_generator}], temperature=1, n=10)

API: {'endpoint': 'https://sweden-central-nlp-llm-instance-01.openai.azure.com/', 'key1': '11f4de001dca4e31ac5b124f71963c01', 'key2': '22ea99d60f844d59b7b7e511c5945b4a', 'region': 'swedencentral', 'model_id': 'llm-chat-v4-128k-deploy-1', 'max_tokens': 4096}


In [276]:
json.dump(
    list(
        set(
            try_json_loads_list(open("data/questions.json", "r").read())
            + list(
                chain(
                    *[
                        try_json_loads_list(remove_backticks(x.message.content))
                        for x in generated_questions.choices
                    ]
                )
            )
        )
    ),
    open("data/questions.json", "w"),
)

In [2]:
import pandas as pd
from pathlib import Path
from helpers.utils import join_lists_from_dir

SPLIT_QUESTIONS_PATH = Path("data/split_questions/")

df = pd.DataFrame(join_lists_from_dir(SPLIT_QUESTIONS_PATH))
topics = df["tasks"].explode().dropna().apply(lambda x: x.split(": ")[0].split(",")).explode().unique().tolist()
df["num_tasks"] = df["tasks"].apply(len)
df[["num_tasks", "can_i_answer"]].value_counts()

num_tasks  can_i_answer
1          False           11881
0          True             1441
2          False             976
0          False             239
1          True               70
3          False              24
2          True               22
4          False               2
Name: count, dtype: int64

In [3]:
df = (
    df
    .query("not (num_tasks == 0 and can_i_answer == False)")
    .query("not (num_tasks >= 1 and can_i_answer == True)")
)

In [51]:
import json
from pathlib import Path

from pydantic import BaseModel
from helpers.utils import join_lists_from_dir
from models.inputs import Function

FUNCTIONS_PATH = Path("data/functions/")

import re

def get_json_list_from_string(s: str) -> list:
    clean_s = re.sub(r"^.*?(\[(.*)\]).*", r"\1", s, flags=re.DOTALL)
    return json.loads(clean_s)


def clean_list(f: str):
    l = json.load(open(f, "r"))
    if len(l) == 1 and type(l[0]) == str:
        l: list[dict] = get_json_list_from_string(l[0])
        print(f"Cleaning {f}...")
    json.dump(l, open(f, "w"), indent=4)

[clean_list(x) for x in FUNCTIONS_PATH.rglob("*.json")]

def try_load_model(model: BaseModel, data: dict):
    try:
        return model(**data)
    except Exception as e:
        print(f"Got error {e!r} when trying to load model {model} with data {data}")
    
functions: list[Function] = [try_load_model(Function, x) for x in join_lists_from_dir(FUNCTIONS_PATH)]

In [54]:
from faisser import FaissDB, Document

docs = [
    Document(
        page_content=f"{x.name}|{x.description}",
        metadata=x.model_dump(mode="json"),
    )
    for x in functions
]

In [80]:
import re
from models.generic import QuestionSplit

resp = """<s> ### Instruction: You function as a reliable and intelligent assistant, adept at simplifying complex queries. When presented with a question in JSON format as input, your task is to produce a JSON object as output. This object should comprise the keys "can_i_answer" (true if you can respond without external resources, false if external resources are indispensable) and "tasks" (a list outlining the steps to answer the input question, applicable when "can_i_answer" is false). Your output should have the following schema: {"properties": {"question": {"title": "Question", "type": "string"}, "tasks": {"items": {"type": "string"}, "title": "Tasks", "type": "array"}, "can_i_answer": {"title": "Can I Answer", "type": "boolean"}}, "required": ["question", "tasks", "can_i_answer"], "title": "QuestionSplit", "type": "object"}</s> \n### Input: \n{'question': 'What is the age of the current Prime Minister of India?            '}\n        \n### Response: \n{"can_i_answer": true, "tasks": []}\n    </s>"""


re.findall(r"### Response:\s*\n?(.*)(?:</s>)", resp, flags=re.DOTALL)[0]

'{"can_i_answer": true, "tasks": []}\n    '

In [9]:
print(resp)

<s> <|system|>
You, as a trustworthy and smart assistant, are responsible for creating sequential steps for achieving goals. No need for actual function execution; simply organize them logically using your cognitive capabilities. The output must adhere strictly to the JSON format.{"$defs": {"ExtractEntry": {"properties": {"id": {"title": "Id", "type": "string"}, "context": {"items": {"type": "string"}, "title": "Context", "type": "array"}, "schema": {"items": {"$ref": "#/$defs/ExtractSchemaEntry"}, "title": "Schema", "type": "array"}}, "required": ["id", "context"], "title": "ExtractEntry", "type": "object"}, "ExtractSchemaEntry": {"properties": {"name": {"title": "Name", "type": "string"}, "type": {"title": "Type", "type": "string"}, "description": {"title": "Description", "type": "string"}}, "required": ["name", "type", "description"], "title": "ExtractSchemaEntry", "type": "object"}, "FunctionEntry": {"properties": {"id": {"title": "Id", "type": "string"}, "name": {"title": "Name", 

In [58]:
from pickler import PickleOperator

PickleOperator().load("function_docs.pkl")

[Document(page_content='MARKET_DATA_RETRIEVAL|Fetches current market data, trends, and statistics from financial markets.', metadata={'name': 'MARKET_DATA_RETRIEVAL', 'description': 'Fetches current market data, trends, and statistics from financial markets.', 'parameters': {'type': 'object', 'properties': [{'name': 'symbols', 'type': 'array', 'description': 'An array of market symbols (e.g., stock tickers, commodity codes) to retrieve data for.'}, {'name': 'data_type', 'type': 'string', 'description': "The specific type of market data to retrieve, such as 'price', 'volume', or 'market_cap'."}], 'required': ['symbols']}}),
 Document(page_content='POLITICAL_ANALYSIS|Provides analysis on political situations, events, or figures based on input parameters.', metadata={'name': 'POLITICAL_ANALYSIS', 'description': 'Provides analysis on political situations, events, or figures based on input parameters.', 'parameters': {'type': 'object', 'properties': [{'name': 'topic', 'type': 'string', 'des

In [60]:
from helpers.utils import get_all_splits
from models.generic import QuestionSplit
from models import inputs, outputs

splits = get_all_splits(r"D:\SmartLLM\data\split_questions")
splits = [QuestionSplit(**x) for x in splits]

In [71]:
from models.inputs import StepsInput
from models.outputs import StepsOutput

q = {"query":"Where does Mark Zuckerberg live and what's the temperature there?","steps":["SEARCH: Find out where Mark Zuckerberg lives","RESEARCH: Determine the current temperature in that location"],"functions":[{"name":"FIND_LOCATION","description":"Locates a geographical position or address based on provided search criteria.","parameters":{"type":"object","properties":[{"name":"search_term","type":"string","description":"The term used to find a specific location or address."},{"name":"precision","type":"string","description":"The desired precision of the location search, such as 'city', 'street', or 'exact'."}],"required":["search_term"]}},{"name":"LOCATION_FINDER","description":"Locates and provides information about a specific place or address.","parameters":{"type":"object","properties":[{"name":"query","type":"string","description":"The search query, which can be a place name, address, or type of location, such as 'restaurant' or 'hospital'."},{"name":"coordinates","type":"object","description":"The geographic coordinates (latitude and longitude) to use as a reference point for the search."},{"name":"radius","type":"number","description":"The search radius in meters around the reference point within which to locate places."}],"required":["query"]}},{"name":"FORBES_DATA_RETRIEVAL","description":"Fetches data from Forbes, such as lists of the wealthiest individuals, top companies, etc.","parameters":{"type":"object","properties":[{"name":"list_type","type":"string","description":"The type of Forbes list to retrieve, e.g., 'billionaires', 'companies', 'celebrities'."},{"name":"year","type":"integer","description":"The year for which to retrieve the Forbes list."}],"required":["list_type"]}},{"name":"GENERATE_CLIMATE_REPORT","description":"Generates a detailed climate report for a specific region including temperature trends, precipitation, and extreme weather events.","parameters":{"type":"object","properties":[{"name":"region","type":"string","description":"The geographical region for which the climate report is to be generated."},{"name":"time_frame","type":"string","description":"The time frame for the climate report, such as 'current', 'historical', or a specific date range."}],"required":["region"]}},{"name":"WEATHER_FORECAST_RETRIEVAL","description":"Retrieves current weather conditions and forecasts for a specified location.","parameters":{"type":"object","properties":[{"name":"location","type":"string","description":"The location for which to retrieve weather information, such as a city name or coordinates."},{"name":"forecast_days","type":"integer","description":"The number of days to retrieve the forecast for."},{"name":"units","type":"string","description":"The units in which to return the weather data, such as 'metric' or 'imperial'. The default is 'metric'."}],"required":["location"]}},{"name":"ENERGY_PRODUCTION_ANALYSIS","description":"Analyzes energy production data from various sources to determine trends and efficiency.","parameters":{"type":"object","properties":[{"name":"energy_type","type":"string","description":"The type of energy source to analyze, such as 'solar', 'wind', 'fossil_fuel'."},{"name":"region","type":"string","description":"The geographic region for which to analyze energy production."},{"name":"time_period","type":"string","description":"The time period over which energy production data should be analyzed."}],"required":["energy_type"]}}]}
StepsInput.model_validate(q)
o = "{\"overview\": \"To identify the current advancements in AI as of 2023, we need to explore recent scientific and technological updates, as well as gather detailed information on specific AI technologies and their current trends.\", \"steps\": [{\"id\": \"step_1\", \"thought\": \"We should start by looking for recent scientific and technological advancements in the field of AI.\", \"explore_tools\": \"Tools that provide updates on scientific and technological innovations can help us find the latest advancements in AI.\", \"available_tools\": [\"SCI_TECH_INNOVATIONS\"], \"choose_tool\": \"The 'SCI_TECH_INNOVATIONS' tool is suitable for identifying recent advancements in AI.\", \"understand_dependencies\": null, \"extract\": [], \"function\": [{\"id\": \"function_1\", \"name\": \"SCI_TECH_INNOVATIONS\", \"parameters\": [{\"name\": \"field\", \"value\": \"Artificial Intelligence\"}, {\"name\": \"date_range\", \"value\": \"2023-01-01 to 2023-12-31\"}, {\"name\": \"keywords\", \"value\": [\"AI\", \"machine learning\", \"neural networks\", \"quantum computing\"]}]}]}, {\"id\": \"step_2\", \"thought\": \"After identifying the advancements, we can use a large language model to summarize the information.\", \"explore_tools\": \"A large language model can help synthesize the information into a coherent summary.\", \"available_tools\": [\"LARGE_LANGUAGE_MODEL\"], \"choose_tool\": \"The 'LARGE_LANGUAGE_MODEL' is suitable for generating a narrative summary of the AI advancements.\", \"understand_dependencies\": null, \"extract\": [], \"function\": [{\"id\": \"function_2\", \"name\": \"LARGE_LANGUAGE_MODEL\", \"parameters\": [{\"name\": \"prompt\", \"value\": \"Summarize the current advancements in AI as of 2023 based on the latest scientific and technological updates.\"}, {\"name\": \"max_tokens\", \"value\": 500}, {\"name\": \"temperature\", \"value\": 0.7}, {\"name\": \"top_p\", \"value\": 1}]}]}]}"
StepsOutput.model_validate_json(o).dict()

C:\Users\balas\AppData\Local\Temp\ipykernel_17496\3742279420.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  StepsOutput.model_validate_json(o).dict()


{'overview': 'To identify the current advancements in AI as of 2023, we need to explore recent scientific and technological updates, as well as gather detailed information on specific AI technologies and their current trends.',
 'steps': [{'id': 'step_1',
   'thought': 'We should start by looking for recent scientific and technological advancements in the field of AI.',
   'explore_tools': 'Tools that provide updates on scientific and technological innovations can help us find the latest advancements in AI.',
   'available_tools': ['SCI_TECH_INNOVATIONS'],
   'choose_tool': "The 'SCI_TECH_INNOVATIONS' tool is suitable for identifying recent advancements in AI.",
   'understand_dependencies': None,
   'extract': [],
   'function': [{'id': 'function_1',
     'name': 'SCI_TECH_INNOVATIONS',
     'parameters': [{'name': 'field', 'value': 'Artificial Intelligence'},
      {'name': 'date_range', 'value': '2023-01-01 to 2023-12-31'},
      {'name': 'keywords',
       'value': ['AI',
        '

In [32]:
from faisser import FaissDB, Document

vdb = FaissDB(filename="functions.pkl")

d:\SmartLLM\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

d:\SmartLLM\env\Lib\site-packages\torch\_utils.py:831: UserWarning:

TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()



In [ ]:
from itertools import chain
from helpers.utils import get_chunks

def get_input_from_split(split: QuestionSplit) -> inputs.StepsInput:
    function_docs = list(chain(
        *[
            vdb.similarity_search_with_relevance_scores(tsk, k=2) 
            for tsk in split.tasks
        ]
    ))
    unique_functions = {
        doc.page_content: inputs.Function(**doc.metadata) 
        for doc, score in function_docs
    }
    return inputs.StepsInput(
        query=split.question,
        steps=split.tasks,
        functions=list(unique_functions.values()),
    )


inputs_schemas = [get_input_from_split(x) for x in splits[:10]]

Shape(name=circle, sides=0)

In [1]:
import pandas as pd

query_tokens_df = pd.read_csv("query_tokens.csv")

In [1]:
from faisser import FaissDB, Document

vdb = FaissDB(filename="functions.pkl")

d:\SmartLLM\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\SmartLLM\env\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
vdb.similarity_search_with_score("Determine which river is currently the longest in India", k=10)

ConnectionError: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: ff31e25b-183f-43ed-8817-d7885a6b4113)')

In [1]:
from pathlib import Path
from models.inputs import StepsInput, BaseModel
from models.outputs import StepsOutput
from helpers.utils import try_json_load, clean_json_str, try_json_loads


DUMP_DIR = Path(r"D:/SmartLLM/data/steps")
step_files = list(DUMP_DIR.rglob("*.json"))
steps: dict[Path, dict|str] = {x: try_json_load(x) for x in DUMP_DIR.rglob("*.json")}

def get_process(p, x: dict) -> StepsOutput:
    try:
        if isinstance(x.get("output"), str):
            return StepsOutput(**try_json_loads(clean_json_str(x.get("output"))))
        return StepsOutput(**x.get("output"))
    except Exception as e:
        print(f"{p}: {e}")

in_out_models: list[tuple[StepsInput, StepsOutput]] = [(StepsInput(**x.get("input")), get_process(p, x)) for p, x in steps.items()]
in_out_dicts = [{"input": x[0].dict(), "output": x[1].dict()} for x in in_out_models if x[1]]
len(in_out_dicts)

d:\SmartLLM\env\Lib\site-packages\pydantic\_internal\_fields.py:184: UserWarning: Field name "schema" shadows an attribute in parent "BaseModel"; 
  warnings.warn(
d:\SmartLLM\env\Lib\site-packages\pydantic\_internal\_fields.py:184: UserWarning: Field name "schema" shadows an attribute in parent "ExtractEntryBase"; 
  warnings.warn(


D:\SmartLLM\data\steps\step_20240125215606_6015008508513764.json: 1 validation error for StepsOutput
steps.1.choose_tool
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type
D:\SmartLLM\data\steps\step_20240126011559_3801004680542258.json: 2 validation errors for StepsOutput
steps.1.choose_tool
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type
steps.2.choose_tool
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type
D:\SmartLLM\data\steps\step_20240126042403_5079862870981426.json: models.outputs.StepsOutput() argument after ** must be a mapping, not list
D:\SmartLLM\data\steps\step_20240127011043_030754399792221943.json: models.outputs.

C:\Users\balas\AppData\Local\Temp\ipykernel_12408\3398263605.py:20: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  in_out_dicts = [{"input": x[0].dict(), "output": x[1].dict()} for x in in_out_models if x[1]]


25548

In [2]:
from itertools import chain

outputs = [_out for _in, _out in in_out_models if _out]
steps = list(chain(*[_out.steps for _out in outputs if _out]))
extracts = [_e for _step in steps for _e in _step.extract]

In [3]:
extract_w_funcs = [out.get_extracts_with_functions() for i, out in enumerate(outputs)]
extract_w_funcs = [x for x in list(chain(*extract_w_funcs))]

In [217]:
import re
from pathlib import Path
from helpers.utils import try_json_load, try_json_loads, clean_json_str

DATA_FILE = Path(r"D:/SmartLLM/data/extract_with_functions.json")
DUMP_DIR = Path(r"D:/SmartLLM/data/output_filled_schema")
DEFAULT_DUMP_PATH = DUMP_DIR / "func_schema.json"

ID_FORMAT = re.compile(r"(func_schema_\d+).*")

def get_generated():
    return {x.stem: try_json_load(x) for x in DUMP_DIR.rglob("*.json")}

def get_extracts_with_functions() -> list[dict]:
    return {x.pop("id"): x for x in try_json_load(DATA_FILE)}

all_extracts = get_extracts_with_functions()
generated = get_generated()

In [218]:
from models.extractor import ExtractorInput

extractor_in_out_dicts = [
    {
        "input": ExtractorInput.model_validate(all_extracts[ID_FORMAT.findall(k)[0]]
                 | {"context": [list(f.values())[0] for f in v.get("function_outputs", [])]}).model_dump(mode="json"),
        "output": v.get("schema_dict", {}),
    }
    for k, v in generated.items()
    if isinstance(v, dict)
]

In [129]:
from datasets import Dataset

dataset_1 = Dataset.from_json("datasets/llm1_dataset.jsonl")
dataset_2 = Dataset.from_json("datasets/llm2_dataset.jsonl")
dataset_3 = Dataset.from_json("datasets/llm3_dataset.jsonl")
all_datasets = {"llm1": dataset_1, "llm2": dataset_2, "llm3": dataset_3}

Generating train split: 25548 examples [00:00, 30057.84 examples/s]


In [138]:
del all_datasets, dataset_1, dataset_3, dataset_2

In [ ]:
"""
<s><<SYS>> You, being an honest and intelligent assistant, are tasked with creating steps for accomplishment. There's no requirement for executing the functions; instead, organize them logically using your cognitive abilities. The output format should be strictly in JSON. <<SYS>> 
[INST] {"query": "What are the latest developments in quantum computing as of 2023?", "steps": ["RESEARCH: Investigate the most recent advancements in quantum computing technology"], "functions": [{"name": "SCI_TECH_INNOVATIONS", "description": "Provides details on recent scientific and technological advancements in various fields.", "parameters": {"type": "object", "properties": [{"name": "field", "type": "string", "description": "The specific field of science or technology to get updates on, such as \'Biotechnology\', \'Artificial Intelligence\', or \'Renewable Energy\'."}, {"name": "date_range", "type": "string", "description": "The range of dates to search for innovations (format: YYYY-MM-DD to YYYY-MM-DD)."}, {"name": "keywords", "type": "array", "description": "A list of keywords to refine the search for innovations."}], "required": ["field"]}}, {"name": "TECHNOLOGY_INFORMATION_RETRIEVAL", "description": "Fetches detailed information about specific technologies, including specifications, applications, and advancements.", "parameters": {"type": "object", "properties": [{"name": "technology_name", "type": "string", "description": "The name of the technology to retrieve information for."}, {"name": "aspect", "type": "string", "description": "The particular aspect of the technology to inquire about, such as \'specifications\', \'history\', or \'current_trends\'."}], "required": ["technology_name"]}}, {"name": "TECHNOLOGY_REPORT_GENERATOR", "description": "Compiles technology reports based on recent developments, trends, and analyses.", "parameters": {"type": "object", "properties": [{"name": "topics", "type": "array", "description": "A list of technology topics to be included in the report."}, {"name": "depth", "type": "string", "description": "The level of depth for the report, either a \'summary\' or a \'detailed\' analysis."}], "required": ["topics"]}}, {"name": "FORMAT_TEXT_DATA", "description": "Formats and structures text data according to specified formatting rules.", "parameters": {"type": "object", "properties": [{"name": "text", "type": "string", "description": "The block of text to be formatted."}, {"name": "format_style", "type": "string", "description": "The desired text formatting style, such as \'plain\', \'markdown\', or \'html\'."}], "required": ["text"]}}]} [/INST]
{"overview": "To uncover the latest developments in quantum computing as of 2023, we must research recent scientific and technological advancements, retrieve in-depth information specific to quantum computing, generate a comprehensive report on these developments, and finally format the data for presentation.", "steps": [{"id": "step_1", "thought": "We should start by identifying the most recent scientific and technological advancements in the field of quantum computing.", "explore_tools": "A tool that provides updates on the latest scientific and technological innovations can be used to acquire this information.", "available_tools": ["SCI_TECH_INNOVATIONS"], "choose_tool": "The \'SCI_TECH_INNOVATIONS\' tool will be used because it specializes in providing the latest updates in specific fields, including quantum computing.", "understand_dependencies": null, "extract": [], "function": [{"id": "function_1", "name": "SCI_TECH_INNOVATIONS", "parameters": [{"name": "field", "value": "Quantum Computing"}, {"name": "date_range", "value": "2023-01-01 to 2023-12-31"}, {"name": "keywords", "value": ["quantum supremacy", "quantum entanglement", "qubit", "quantum algorithm"]}]}]}, {"id": "step_2", "thought": "After identifying recent innovations, we should gather detailed information specific to the advancements in quantum computing.", "explore_tools": "A tool that retrieves detailed information about specific technologies can be used to delve deeper into quantum computing advancements.", "available_tools": ["TECHNOLOGY_INFORMATION_RETRIEVAL"], "choose_tool": "The \'TECHNOLOGY_INFORMATION_RETRIEVAL\' tool will be selected to get comprehensive details on the latest developments in quantum computing.", "understand_dependencies": null, "extract": [], "function": [{"id": "function_2", "name": "TECHNOLOGY_INFORMATION_RETRIEVAL", "parameters": [{"name": "technology_name", "value": "Quantum Computing"}, {"name": "aspect", "value": "current_trends"}]}]}, {"id": "step_3", "thought": "Now it\'s time to compile a report based on the information gathered from the research and in-depth analysis of quantum computing advancements.", "explore_tools": "A report generator can be used to compile a structured report summarizing the recent developments in quantum computing.", "available_tools": ["TECHNOLOGY_REPORT_GENERATOR"], "choose_tool": "The \'TECHNOLOGY_REPORT_GENERATOR\' tool will be used to create a comprehensive report on the latest developments in quantum computing.", "understand_dependencies": null, "extract": [], "function": [{"id": "function_3", "name": "TECHNOLOGY_REPORT_GENERATOR", "parameters": [{"name": "topics", "value": ["Quantum Computing"]}, {"name": "depth", "value": "detailed"}]}]}, {"id": "step_4", "thought": "Finally, we should format the report to ensure it is clearly presented and easy to read.", "explore_tools": "A text formatting tool can be employed to structure and present the report according to standard formatting styles.", "available_tools": ["FORMAT_TEXT_DATA"], "choose_tool": "The \'FORMAT_TEXT_DATA\' tool will be used to format the report in a reader-friendly style, such as markdown or html.", "understand_dependencies": null, "extract": [], "function": [{"id": "function_4", "name": "FORMAT_TEXT_DATA", "parameters": [{"name": "text", "value": "{{step_3.function.function_3.output}}"}, {"name": "format_style", "value": "markdown"}]}]}]} </s>
"""

In [144]:
from datasets import Dataset

D_PATH = "datasets/llm3_dataset.jsonl"
dataset = Dataset.from_json(D_PATH)

dataset_lst = [
    {
        "conversations": [
            {
                "from": "human"
                if m["role"] == "user"
                else "gpt"
                if m["role"] == "assistant"
                else "system",
                "value": m["content"],
            }
            for m in x["messages"]
        ]
    }
    for x in dataset
]
Dataset.from_list(dataset_lst).to_json("datasets/llm3_sharegpt.jsonl")
# dataset_lst

Creating json from Arrow format: 100%|██████████| 8/8 [00:00<00:00, 24.51ba/s]


15310476

In [133]:
from transformers import AutoTokenizer

MODEL_NAME = "mistralai/Mistral-7B-v0.1"

class ModelMessages:
    def __init__(self, mode_name: str = "HuggingFaceH4/zephyr-7b-beta"):
        self.tokenizer = AutoTokenizer.from_pretrained(mode_name)
    
    def model_format_single(self, messages: list[dict[str, str]]) -> str:
        try:
            return self.tokenizer.apply_chat_template(messages, tokenize=False)
        except Exception as e:
            print(e)
            return self.tokenizer.apply_chat_template(messages[1:], tokenize=False)

    def format_dataset_messages(self, messages_dict: dict[str, list[dict[str, str]]]):
        messages = messages_dict.get("messages", [])
        return {"messages": self.model_format_single(messages)}

    def model_format(self, messages_list: list[list[dict[str, str]]]):
        try:
            self.model_format_single(messages_list[0])
        except Exception as e1:
            try:
                self.model_format_single(messages_list[0][1:])
                messages_list=[x[1:] for x in messages_list]
            except Exception as e2:
                print(e1)
                print(e2)
                return []
        return [self.model_format_single(x) for x in messages_list]

# model_messages = {k: model_format(v) for k, v in dataset_messages.items()}

In [135]:
ds = dataset_2.map(ModelMessages("mistralai/Mistral-7B-v0.1").format_dataset_messages)

ds[0]

Map: 100%|██████████| 25548/25548 [00:10<00:00, 2369.89 examples/s]


{'messages': '<s>[INST] <<SYS>>\nYou, being an honest and intelligent assistant, are tasked with creating steps for accomplishment. There\'s no requirement for executing the functions; instead, organize them logically using your cognitive abilities. The output format should be strictly in JSON.\n<</SYS>>\n\n{"query": "What are the latest developments in quantum computing as of 2023?", "steps": ["RESEARCH: Investigate the most recent advancements in quantum computing technology"], "functions": [{"name": "SCI_TECH_INNOVATIONS", "description": "Provides details on recent scientific and technological advancements in various fields.", "parameters": {"type": "object", "properties": [{"name": "field", "type": "string", "description": "The specific field of science or technology to get updates on, such as \'Biotechnology\', \'Artificial Intelligence\', or \'Renewable Energy\'."}, {"name": "date_range", "type": "string", "description": "The range of dates to search for innovations (format: YYYY

In [350]:
import json
from random import choices

l = [[y['content'] for y in x['messages'][1:]] for x in choices(dataset_3, k=3)]
[json.dumps({"input": json.loads(x[0]), "output": json.loads(x[1])}) for x in l]

['{"input": {"schema": [{"name": "book_titles", "type": "array", "description": "A list of the top selling books according to the New York Times Best Seller list for 2023."}], "context": [{"generated_text": ["The Midnight Library by Matt Haig", "The Vanishing Half by Brit Bennett", "Where the Crawdads Sing by Delia Owens", "The Four Winds by Kristin Hannah", "Atomic Habits by James Clear"]}]}, "output": {"book_titles": ["The Midnight Library by Matt Haig", "The Vanishing Half by Brit Bennett", "Where the Crawdads Sing by Delia Owens", "The Four Winds by Kristin Hannah", "Atomic Habits by James Clear", "The Invisible Life of Addie LaRue by V.E. Schwab", "Malibu Rising by Taylor Jenkins Reid", "The Last Thing He Told Me by Laura Dave", "Project Hail Mary by Andy Weir", "The Lincoln Highway by Amor Towles"]}}',
 '{"input": {"schema": [{"name": "variants", "type": "array", "description": "List of the new COVID-19 variants identified."}], "context": [{"update": "As of the recent surveillanc

In [ ]:
from models.extractor import ExtractorInput
from datasets import Dataset

[ExtractorInput.model_validate_json(x).model_dump() for x in Dataset.from_json("llm3_test.jsonl")['input']]

In [111]:
import json


def convert_messages_to_system_input_output(data):
    messages = data["messages"]
    return {
        "system": messages[0]["content"],
        "input": json.loads(messages[1]["content"]),
        "output": json.loads(messages[2]["content"]),
    }

def convert_messages_to_system_input_output_wo_json_loads(data):
    messages = data["messages"]
    return {
        "system": messages[0]["content"],
        "input": messages[1]["content"],
        "output": messages[2]["content"],
    }


def convert_system_input_output_to_messages(data):
    return {
        "messages": [
            {"role": "system", "content": data["system"]},
            {"role": "user", "content": json.dumps(data["input"])},
            {"role": "assistant", "content": json.dumps(data["output"])},
        ]
    }




dataset_1_sio = dataset_1.map(convert_messages_to_system_input_output).remove_columns(
    ["messages"]
)

In [100]:
import re
import json
from pathlib import Path
import pandas as pd

DUMP_DIR = Path(r"D:/SmartLLM/data/question_answers")

df = pd.DataFrame([json.load(open(x)) for x in DUMP_DIR.rglob("*.json")])

unanswered_regex = re.compile(
    r"^(To\s*obtain\s*the\s*most|I\s*am\s*unable\s*to|I'm\s*unable\s*to|I\s*cannot\s*provide|For\s*the\s*most|For\s*the\s*newest|For\s*the\s*latest|To\s*find\s*the\s*most|As\s*of\s*my\s*knowledge|As\s*a\s*language\s*model|I'm\s*sorry|I\s*do\s*not|As\s*an\s*AI|To\s*obtain\s*the\s*latest)",
    flags=re.IGNORECASE | re.DOTALL,
)

ans_truths = df.apply(lambda x: not unanswered_regex.search(x["answer"]), axis=1)
df = df[ans_truths].reset_index(drop=True)
df_org = pd.DataFrame(dataset_1_sio.to_list())
df_org = df_org.set_index(df_org["input"].apply(lambda x: x["question"]))
df = df.set_index(["question"])
merged_df = df_org.merge(df, left_index=True, right_index=True, how="outer")
merged_df["answer"].fillna("", inplace=True)
merged_df.reset_index(drop=True, inplace=True)
merged_df["output"] = (
    merged_df[["output", "answer"]]
    .apply(lambda x: x["output"] | {"response": x["answer"]}, axis=1)
    .to_list()
)
merged_df["can_i_answer"] = merged_df["output"].apply(
    lambda x: x.get("can_i_answer", False)
)
merged_df = pd.DataFrame(
    [
        x
        | {
            "output": x["output"]
            | {
                "can_i_answer": True if x["output"]["response"] else False,
                "response": x["output"]["response"]
                if x["output"]["response"]
                else None,
            },
            "can_i_answer": True if x["output"]["response"] else False,
        }
        for x in merged_df.to_dict(orient="records")
    ]
)

In [101]:
merged_df.iloc[0].to_dict()

{'system': 'Embodying the role of an earnest and brainy assistant who can reframe questions into actionable components, you face a question framed in JSON. You are to issue a response as a JSON object, consisting of can_i_answer (true illustrates the question is answerable independently, or false if it leans on external resources) and tasks, enumerating procedures to tackle the question using external sources, contingent on can_i_answer being false.',
 'input': {'question': '2023 Nobel Prize winners in Physics announced?'},
 'output': {'can_i_answer': False,
  'tasks': ['SEARCH: Check for the announcement of Nobel Prize winners in Physics for 2023'],
  'response': None},
 'answer': '',
 'can_i_answer': False}

In [102]:
dst = Dataset.from_list(
    list({
        f"{x['messages'][0]['content']}|{json.loads(x['messages'][1]['content'])['question']}|{json.loads(x['messages'][2]['content'])['can_i_answer']}|{json.loads(x['messages'][2]['content'])['response']}|{json.loads(x['messages'][2]['content'])['tasks']}": x
        for x in merged_df.apply(
            convert_system_input_output_to_messages, axis=1
        ).to_list()
    }.values())
)

In [104]:
dst_d = dst.map(convert_messages_to_system_input_output).to_list()

Map: 100%|██████████| 14358/14358 [00:01<00:00, 7635.97 examples/s]


In [108]:
dst.to_json("datasets/llm1_dataset.jsonl")

Creating json from Arrow format: 100%|██████████| 15/15 [00:00<00:00, 53.14ba/s]


12248457

In [127]:
mm = ModelMessages(MODEL_NAME)

In [132]:
dst.map(mm.format_dataset_messages).to_list()

Map: 100%|██████████| 14358/14358 [00:03<00:00, 4454.51 examples/s]


[{'messages': '<|system|>\nEmbodying the role of an earnest and brainy assistant who can reframe questions into actionable components, you face a question framed in JSON. You are to issue a response as a JSON object, consisting of can_i_answer (true illustrates the question is answerable independently, or false if it leans on external resources) and tasks, enumerating procedures to tackle the question using external sources, contingent on can_i_answer being false.</s>\n<|user|>\n{"question": "2023 Nobel Prize winners in Physics announced?"}</s>\n<|assistant|>\n{"can_i_answer": false, "tasks": ["SEARCH: Check for the announcement of Nobel Prize winners in Physics for 2023"], "response": null}</s>\n'},
 {'messages': '<|system|>\nYour role as an upright and sharp-witted assistant who can distill questions into simple procedural steps involves handling an input question in JSON. You must deliver a JSON object in response, equipped with the keys can_i_answer (indicating true if the question

In [114]:
pd.DataFrame([convert_messages_to_system_input_output_wo_json_loads(x) for x in dst])

,system,input,output
0,Embodying the role of an earnest and brainy as...,"{""question"": ""2023 Nobel Prize winners in Phys...","{""can_i_answer"": false, ""tasks"": [""SEARCH: Che..."
1,Your role as an upright and sharp-witted assis...,"{""question"": ""2023 advancements in aerospace t...","{""can_i_answer"": false, ""tasks"": [""SEARCH,SCIE..."
2,Taking on the responsibility as an honest and ...,"{""question"": ""2023 advancements in artificial ...","{""can_i_answer"": false, ""tasks"": [""SEARCH: Loo..."
3,You are an honest and smart assistant who can ...,"{""question"": ""2023 advancements in artificial ...","{""can_i_answer"": false, ""tasks"": [""SEARCH: Fin..."
4,Your duty as a sincere and intelligent assista...,"{""question"": ""2023 advancements in cancer trea...","{""can_i_answer"": false, ""tasks"": [""RESEARCH: S..."
...,...,...,...
14353,In your capacity as a sincere and intelligent ...,"{""question"": ""Winner of the 2023 Nobel Peace P...","{""can_i_answer"": false, ""tasks"": [""SEARCH: Fin..."
14354,In your capacity as a sincere and intelligent ...,"{""question"": ""Winner of the 2023 Nobel Peace P...","{""can_i_answer"": false, ""tasks"": [""SEARCH: Loo..."
14355,Serving as an authentic and insightful assista...,"{""question"": ""Winners of the 2023 Academy Awar...","{""can_i_answer"": false, ""tasks"": [""SEARCH,NEWS..."
14356,In your capacity as a sincere and intelligent ...,"{""question"": ""Youngest entrepreneur on the For...","{""can_i_answer"": false, ""tasks"": [""SEARCH: Loo..."


In [118]:
import pickle

pickle.load(open("breakdown_dataset.pkl", "rb"))

,question,tasks,can_i_answer,num_tasks
0,What are the latest developments in quantum co...,[RESEARCH: Investigate the most recent advance...,False,1
1,Recent sports records broken in the first quar...,"[SEARCH,NEWS: Look up recent sports achievemen...",False,1
2,What are the latest scientific discoveries in ...,[RESEARCH: Explore the most recent findings an...,False,1
3,How has the pandemic affected education system...,[ANALYSIS: Study the impact of the pandemic on...,False,1
4,Who are the biggest influencers in technology ...,"[SEARCH,NEWS: Identify key technology influenc...",False,1
...,...,...,...,...
14650,Give me advice on maintaining a balanced diet ...,[],True,0
14651,Provide workout routines for effective weight ...,[],True,0
14652,Explain the core principles behind quantum mec...,[],True,0
14653,What are the key concepts in machine learning ...,[],True,0


In [135]:
print(mm.model_format_single(
    [
        {"role": "system", "content": "A"},
        {"role": "user", "content": "B"},
        {"role": "assistant", "content": ""},
    ]
))

<|system|>
A</s>
<|user|>
B</s>
<|assistant|>
</s>



In [141]:
steps = OutModel.parse_file("D:\SmartLLM\whatisthecurrenttemperatureinla_20240205082755_9951052316583368.json").steps_output.steps

C:\Users\balas\AppData\Local\Temp\ipykernel_26788\2631307264.py:1: PydanticDeprecatedSince20: The `parse_file` method is deprecated; load the data from file, then if your data is JSON use `model_validate_json`, otherwise `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  steps = OutModel.parse_file("D:\SmartLLM\whatisthecurrenttemperatureinla_20240205082755_9951052316583368.json").steps_output.steps


In [4]:
context = {
    "step_1": {
        "function": {
            "function_1": {
                "input": {"location": "Los Angeles", "period": "current"},
                "function_name": "WEATHER_FORECAST",
                "output": {
                    "location": "Los Angeles",
                    "period": "current",
                    "weather_forecast": {
                        "temperature": "22°C",
                        "condition": "Sunny",
                        "humidity": "59%",
                        "wind": "5 mph NW",
                        "visibility": "10 miles",
                        "pressure": "1015 hPa",
                    },
                },
            }
        }
    },
    "step_2": {
        "function": {
            "function_2": {
                "input": {
                    "text": "The current temperature in Los Angeles is None.",
                    "format_style": "plain",
                },
                "function_name": "FORMAT_TEXT_DATA",
                "output": 'The context provided does not specify the current temperature in Los Angeles as it states "None." To obtain the current temperature, you would need to check a reliable weather service or website for the most up-to-date information.',
            }
        },
        "extract": {"extract_1": {"temperature": 22}},
    },
}

In [11]:
[
    type_data
    for step_id, step_data in context.items()
    for type_name, t_d in step_data.items()
    for type_id, type_data in t_d.items()
    if type_name == "function"
]

[{'input': {'location': 'Los Angeles', 'period': 'current'},
  'function_name': 'WEATHER_FORECAST',
  'output': {'location': 'Los Angeles',
   'period': 'current',
   'weather_forecast': {'temperature': '22°C',
    'condition': 'Sunny',
    'humidity': '59%',
    'wind': '5 mph NW',
    'visibility': '10 miles',
    'pressure': '1015 hPa'}}},
 {'input': {'text': 'The current temperature in Los Angeles is None.',
   'format_style': 'plain'},
  'function_name': 'FORMAT_TEXT_DATA',
  'output': 'The context provided does not specify the current temperature in Los Angeles as it states "None." To obtain the current temperature, you would need to check a reliable weather service or website for the most up-to-date information.'}]

In [23]:
list(list(context.values())[-1].get("function", {"": {"output": ""}}).values())[-1]['output']

'The context provided does not specify the current temperature in Los Angeles as it states "None." To obtain the current temperature, you would need to check a reliable weather service or website for the most up-to-date information.'

In [52]:
import json
from models.inputs import Function
from faisser import FaissDB, Document


funcs = json.load(open("custom_funcs.json", "r"))
funcs = [Function.model_validate(x) for x in funcs]
docs = [Document(page_content=f"{x.name}|{x.description}", metadata=x.model_dump(mode="json")) for x in funcs]
vdb = FaissDB(filename="custom_funcs.pkl", documents=docs)

In [57]:
vdb.similarity_search("find what is the latest news on qxlab ai?")

[Document(page_content='LLM|A large language model capable of understanding and generating human-like text based on input.', metadata={'name': 'LLM', 'description': 'A large language model capable of understanding and generating human-like text based on input.', 'parameters': {'type': 'object', 'properties': [{'name': 'text', 'type': 'string', 'description': 'The input text to be processed or generated.'}, {'name': 'context', 'type': 'string', 'description': 'Optional context to provide additional information or constraints for text generation.'}, {'name': 'temperature', 'type': 'number', 'description': 'The temperature parameter controlling the randomness of text generation. Default is 1.0.'}, {'name': 'max_tokens', 'type': 'number', 'description': 'The maximum number of tokens (words or subwords) to generate. Default is 50.'}, {'name': 'top_p', 'type': 'number', 'description': 'The top-p (nucleus) sampling parameter for text generation. Default is 1.0.'}], 'required': ['text']}, 'con

In [58]:
vdb = FaissDB(filename="vectorstore.pkl")

In [60]:
d = vdb.similarity_search("", k=1000)

In [1]:
from models.outputs import Step

steps = [
    {
        "id": "step_1",
        "thought": "To obtain the current temperature in Los Angeles, we should look for the latest weather forecasts and data.",
        "explore_tools": "We need a tool that can provide us with the latest weather forecasts and data.",
        "available_tools": [
            "WEATHER_FORECAST_RETRIEVAL",
            "WEATHER_FORECAST",
            "WEATHER_FORECAST",
        ],
        "choose_tool": "The 'WEATHER_FORECAST' tool is suitable for retrieving the latest temperature data for a specific location.",
        "understand_dependencies": None,
        "extract": [
            {
                "id": "extract_1",
                "context": ["step_1.function.function_1"],
                "schema": [
                    {
                        "name": "temperature",
                        "type": "number",
                        "description": "The current temperature in Los Angeles.",
                    }
                ],
            }
        ],
        "function": [
            {
                "id": "function_1",
                "name": "WEATHER_FORECAST",
                "parameters": [
                    {"name": "location", "value": "Los Angeles"},
                    {"name": "period", "value": "current"},
                ],
            }
        ],
    },
    {
        "id": "step_2",
        "thought": "Once we have the temperature data, we should present it in a clear and concise manner.",
        "explore_tools": "We need a tool that can help us format the retrieved data.",
        "available_tools": ["FORMAT_TEXT_DATA"],
        "choose_tool": "The 'FORMAT_TEXT_DATA' tool will be used to format the temperature data into a readable format.",
        "understand_dependencies": "The 'FORMAT_TEXT_DATA' tool will need the raw data from 'step_1.function.function_1' to format the text appropriately.",
        "extract": [
            {
                "id": "extract_2",
                "context": ["step_1.function.function_1", "step_1.extract.extract_1.temperature"],
                "schema": [
                    {
                        "name": "temperature",
                        "type": "number",
                        "description": "The current temperature in Los Angeles.",
                    }
                ],
            }
        ],
        "function": [
            {
                "id": "function_2",
                "name": "FORMAT_TEXT_DATA",
                "parameters": [
                    {
                        "name": "text",
                        "value": "The current temperature in Los Angeles is {{step_2.extract.extract_1.temperature}}.",
                    },
                    {"name": "format_style", "value": "plain"},
                ],
            }
        ],
    }
]

steps = [Step.model_validate(x) for x in steps]

In [4]:
steps[0].model_dump(mode="json", by_alias=True)

{'id': 'step_1',
 'thought': 'To obtain the current temperature in Los Angeles, we should look for the latest weather forecasts and data.',
 'explore_tools': 'We need a tool that can provide us with the latest weather forecasts and data.',
 'available_tools': ['WEATHER_FORECAST_RETRIEVAL',
  'WEATHER_FORECAST',
  'WEATHER_FORECAST'],
 'choose_tool': "The 'WEATHER_FORECAST' tool is suitable for retrieving the latest temperature data for a specific location.",
 'understand_dependencies': None,
 'extract': [{'id': 'extract_1',
   'schema': [{'name': 'temperature',
     'type': 'number',
     'description': 'The current temperature in Los Angeles.'}],
   'context': ['step_1.function.function_1']}],
 'function': [{'id': 'function_1',
   'name': 'WEATHER_FORECAST',
   'parameters': [{'name': 'location', 'value': 'Los Angeles'},
    {'name': 'period', 'value': 'current'}]}]}